# 5008 Project

In [1]:
import pandas as pd

# fb-pages-media.edges is too large for my computer (waiting for testing)
df = pd.read_csv("dataset/fb-pages-food.edges", names=["node_1", "node_2"]) 
df.head(10)

,node_1,node_2
0,0,276
1,0,58
2,0,132
3,0,603
4,0,398
5,0,555
6,1,265
7,1,611
8,2,265
9,2,182


In [2]:
import networkx as nx
from node2vec import Node2Vec

# transform dataframe to networkx object
graph = nx.from_pandas_edgelist(df, "node_1", "node_2")

In [3]:
num_nodes = graph.number_of_nodes() # the number of nodes
num_edges = graph.number_of_edges() # the number of edges
print("num_nodes = ", num_nodes)
print("num_edges = ", num_edges)

num_nodes =  620
num_edges =  2102


## Find edges which can be removed from original graph

When removing the edge, we should make sure that the graph is connected and no redution in the number of nodes. In this step, we store all the omissiable edges in list `omissible_edges` and remove the edges from the original graph.

In [4]:
import numpy as np

rnd = np.random.RandomState()
edges_index = rnd.permutation(num_edges) # generate a permutation array from 0 to num_edges
edges = list(graph.edges())
omissible_edges = list() # the edges that can be removed
t = 0

# Traversal all edges
# If using fb-pages-media.edges, there are totally 206259 edges so it'll take some time to traversal all edges, about 30 min for my computer
for i in edges_index:
    edge = edges[i]
    node_1 = edge[0]
    node_2 = edge[1]
    
    if t % 50 == 0:
        print("processing: %s/%s" %(t,num_edges))
    
    graph.remove_edge(*edge)
    reachable_from_node_1 = nx.connected._plain_bfs(graph, node_1)
    
    if node_2 not in reachable_from_node_1:
        # the drop would make the graph become unconnected or reduce the number of nodes
        # so we shouldn't drop this edge, re-adding the edge
        graph.add_edge(*edge)
    else:
        # the edge is omissible and as a positive sample
        omissible_edges.append(edge)
    t += 1

processing: 0/2102
processing: 50/2102
processing: 100/2102
processing: 150/2102
processing: 200/2102
processing: 250/2102
processing: 300/2102
processing: 350/2102
processing: 400/2102
processing: 450/2102
processing: 500/2102
processing: 550/2102
processing: 600/2102
processing: 650/2102
processing: 700/2102
processing: 750/2102
processing: 800/2102
processing: 850/2102
processing: 900/2102
processing: 950/2102
processing: 1000/2102
processing: 1050/2102
processing: 1100/2102
processing: 1150/2102
processing: 1200/2102
processing: 1250/2102
processing: 1300/2102
processing: 1350/2102
processing: 1400/2102
processing: 1450/2102
processing: 1500/2102
processing: 1550/2102
processing: 1600/2102
processing: 1650/2102
processing: 1700/2102
processing: 1750/2102
processing: 1800/2102
processing: 1850/2102
processing: 1900/2102
processing: 1950/2102
processing: 2000/2102
processing: 2050/2102
processing: 2100/2102


In [5]:
num_omissible_edges = len(omissible_edges)
print("num_omissible_edges = ", num_omissible_edges)

num_omissible_edges =  1483


In [6]:
num_edges_pro = graph.number_of_edges()
num_nodes_pro = graph.number_of_nodes()
print("num_edges after processing = ", num_edges_pro)
print("num_nodes after processing = ", num_nodes_pro)

num_edges after processing =  619
num_nodes after processing =  620


## Implement node2vec on the graph which has been removed some edges

In [7]:
from node2vec import Node2Vec

# Precompute probabilities and generate walks
node2vec = Node2Vec(graph, dimensions=128, walk_length=30, num_walks=200, workers=2)

# Embbed nodes
# If using fb-pages-media.edges, the training time is about 10 min for my computer
model = node2vec.fit(window=10, min_count=1, batch_words=4)

Computing transition probabilities: 100%|█████████████████████████████████████████| 620/620 [00:00<00:00, 10191.28it/s]


In [111]:
# Save embeddings for later use
# model.wv.save_word2vec_format("model/node2vec.wv")

# Save model for later use
# model.save("model/node2vec.model")

In [8]:
node_vec = model.wv
node_vec["0"]

array([-0.25691   ,  0.59746414,  0.41822115,  2.332754  ,  2.0844984 ,
       -2.5893075 , -0.20680542, -2.1315417 , -1.1664872 ,  1.4086227 ,
       -1.5666256 , -3.125769  ,  0.92164975, -0.3960764 ,  1.0003132 ,
        1.018495  , -0.48017418, -1.8804253 , -1.7820462 , -2.0834851 ,
        2.3688147 ,  0.99026257,  1.936604  , -0.62905616,  0.6467913 ,
       -1.6854614 ,  0.714615  ,  2.3905463 ,  0.6351496 ,  1.62053   ,
       -3.4580696 , -0.9160007 , -0.5709128 ,  0.7004316 ,  2.0210693 ,
       -2.9290051 ,  0.00658981,  3.0201344 , -0.3988419 , -0.9816801 ,
       -0.63056415, -0.41915756, -0.3334948 ,  0.03150539,  1.325237  ,
       -1.5036391 ,  1.2572906 ,  0.9132987 ,  0.04754774,  2.020887  ,
        0.7498822 ,  3.0544705 ,  0.18632732,  1.0506004 , -1.8793436 ,
        3.2366354 , -1.6584482 ,  0.5030489 , -0.18581922,  0.95150006,
        1.3536414 , -0.13187096,  0.32911956, -1.1597457 ,  0.69437766,
        0.02385478, -0.3030495 , -1.9419149 , -1.0129371 ,  2.66

## Generate positive samples and negative samples

The guide uses `(node_vec[node_1] + node_vec[node_2])` to embed edges. Here I first use HadamardEmbedder to embed edges, train and do link prediction because I find this edges embedded method performs better than the guide's. Then I'll also try the guide method.

In [144]:
from node2vec.edges import HadamardEmbedder

# Generate the edges embeddings i.e. using vector to express edge
edges_embs = HadamardEmbedder(keyed_vectors=node_vec)
edges_embs[("0", "1")] # The vector of edge from node 0 to node 1

array([ 2.38767952e-01,  6.81317449e-01, -3.77883196e-01,  4.89470291e+00,
        3.29655361e+00,  5.42832851e+00,  1.13847107e-01,  7.29921818e-01,
        4.65948522e-01,  6.33592546e-01, -4.56866771e-01, -6.60342598e+00,
        1.87487662e+00, -1.17829180e+00, -1.18035412e+00,  1.07600224e+00,
        6.64768398e-01, -3.76601160e-01, -1.97623241e+00, -6.76620436e+00,
        3.47384095e+00,  2.48716809e-02, -2.44281793e+00, -5.96328437e-01,
        2.07273275e-01, -2.86231875e+00,  9.06003356e-01, -3.18255067e-01,
        1.17374726e-01, -2.11860085e+00,  2.85964823e+00, -8.75065804e-01,
        6.36873960e-01, -7.18757272e-01, -1.27157056e+00,  4.47820997e+00,
        4.18598950e-03, -4.10882044e+00,  3.37369323e-01,  1.23253369e+00,
        5.12980938e-01, -3.63147169e-01,  1.76511303e-01,  5.75989224e-02,
        1.05924380e+00, -2.25178790e+00, -1.59872234e+00,  5.43805480e-01,
       -4.04455028e-02,  4.54599190e+00,  3.30456614e-01, -1.13594985e+00,
        1.67768449e-01, -

In [89]:
X_pos = [edges_embs[str(i), str(j)] for i,j in omissible_edges]
y_pos = [1] * len(X_pos)

In [109]:
# Create the adjacency matrix of graph
crosstab = pd.crosstab(df.node_1, df.node_2)
idx = crosstab.columns.union(crosstab.index)
adjmat = crosstab.reindex(index=idx, columns=idx, fill_value=0)
adjmat = adjmat.values.T + adjmat.values

In [136]:
num_follow_myself = 0
for i in range(num_nodes):
    for j in range(num_nodes):
        if adjmat[i][j] == 2 and i == j:
            num_follow_myself += 1
num_follow_myself

11

As you can see, it is a square matrix. Now, we will traverse the adjacency matrix to find the positions of the zeros. Please note that we don’t have to go through the entire matrix. The values in the matrix are the same above and below the diagonal, as you can see below:

![adjacency matrix](img/adjmat.webp)

We can either search through the values above the diagonal (green part) or the values below (red part). Let’s search the **values above the diagonal** for zero:

In [137]:
unconnected_edges = list()
# Only search the green part i.e. values above the diagonal to find all unconnected edges
for i in range(0, num_nodes):
    if i % 50 == 0:
        print("processing: %s/%s" % (i,num_nodes))
    for j in range(i+1, num_nodes):
        if adjmat[i][j] == 0:
            unconnected_edges.append((i,j))

processing: 0/620
processing: 50/620
processing: 100/620
processing: 150/620
processing: 200/620
processing: 250/620
processing: 300/620
processing: 350/620
processing: 400/620
processing: 450/620
processing: 500/620
processing: 550/620
processing: 600/620


In [114]:
print(adjmat[0][58])
print(adjmat[58][0])

1
1


In [138]:
# Generate nagtive samples
X_neg = [edges_embs[str(i), str(j)] for i,j in unconnected_edges]
y_neg = [0] * len(X_neg)

In [139]:
X = X_pos + X_neg
y = y_pos + y_neg

print("number of X positive = ", len(X_pos))
print("number of X negtive = ", len(X_neg))
print("number of X = ", len(X))

number of X positive =  1483
number of X negtive =  189799
number of X =  191282


In [140]:
from sklearn.linear_model import LogisticRegression

clfLR = LogisticRegression(random_state=0)
clfLR.fit(X, y)

C:\Users\yelbee\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=0, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [22]:
df.head(10)

,node_1,node_2
0,0,276
1,0,58
2,0,132
3,0,603
4,0,398
5,0,555
6,1,265
7,1,611
8,2,265
9,2,182


## Link prediction

In [141]:
def predict(i, j, classifier):
    return classifier.predict([edges_embs[str(i), str(j)]]), classifier.predict_proba([edges_embs[str(i), str(j)]])


for i in range(0, num_nodes):
    for j in range(i+1, num_nodes):
        ret = predict(i, j, clfLR)
        if ret[0][0] == 1:
            print((i, j), ret[1][0])

(0, 113) [0.20083011 0.79916989]
(0, 276) [0.36839805 0.63160195]
(0, 319) [0.40203293 0.59796707]
(7, 383) [0.44873952 0.55126048]
(9, 46) [0.45316117 0.54683883]
(9, 317) [0.4610523 0.5389477]
(11, 485) [0.38192438 0.61807562]
(22, 187) [0.19789075 0.80210925]
(31, 142) [0.02693867 0.97306133]
(31, 357) [0.38580247 0.61419753]
(31, 424) [0.4674367 0.5325633]
(31, 518) [0.41413218 0.58586782]
(31, 595) [0.01257725 0.98742275]
(40, 185) [0.06336578 0.93663422]
(40, 240) [0.21672477 0.78327523]
(40, 356) [0.47141427 0.52858573]
(52, 319) [0.33170063 0.66829937]
(56, 603) [0.4458831 0.5541169]
(63, 142) [0.1410496 0.8589504]
(63, 164) [0.49781261 0.50218739]
(65, 333) [0.33214702 0.66785298]
(65, 356) [0.28563351 0.71436649]
(67, 351) [0.28334424 0.71665576]
(70, 550) [0.48047816 0.51952184]
(71, 539) [0.49261546 0.50738454]
(81, 225) [0.29113628 0.70886372]
(90, 131) [0.22020137 0.77979863]
(90, 265) [0.24369373 0.75630627]
(90, 351) [0.38409849 0.61590151]
(101, 400) [0.25540446 0.7445

## Use edges embedded method in guide

In [142]:
X_pos_g = [node_vec[str(i)] + node_vec[str(j)] for i,j in omissible_edges]
y_pos_g = [1] * len(X_pos)

X_neg_g = [node_vec[str(i)] + node_vec[str(j)] for i,j in unconnected_edges]
y_neg_g = [0] * len(X_neg)

X_g = X_pos_g + X_neg_g
y_g = y_pos_g + y_neg_g

clfLR_g = LogisticRegression(random_state=0).fit(X_g, y_g)

C:\Users\yelbee\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [143]:
for i in range(0, num_nodes):
    for j in range(i+1, num_nodes):
        ret = predict(i, j, clfLR_g)
        if ret[0][0] == 1:
            print((i,j), ret[1][0])

(0, 20) [0.4014468 0.5985532]
(0, 57) [0.30536144 0.69463856]
(0, 58) [0.10309992 0.89690008]
(0, 96) [0.41557152 0.58442848]
(0, 116) [0.08676905 0.91323095]
(0, 130) [0.49065591 0.50934409]
(0, 151) [0.23699407 0.76300593]
(0, 216) [0.33026935 0.66973065]
(0, 300) [0.38498849 0.61501151]
(0, 341) [0.46538621 0.53461379]
(0, 350) [0.00127953 0.99872047]
(0, 383) [0.21498261 0.78501739]
(0, 418) [0.1107345 0.8892655]
(0, 432) [0.3941621 0.6058379]
(0, 433) [0.40515765 0.59484235]
(0, 478) [0.40188519 0.59811481]
(0, 537) [0.36345352 0.63654648]
(1, 151) [0.17188176 0.82811824]
(1, 508) [0.36050072 0.63949928]
(2, 183) [0.10242597 0.89757403]
(2, 547) [0.49453441 0.50546559]
(2, 550) [0.35065774 0.64934226]
(2, 605) [0.46758136 0.53241864]
(3, 142) [0.39922892 0.60077108]
(3, 179) [0.39478538 0.60521462]
(7, 62) [0.44228982 0.55771018]
(7, 119) [0.48629876 0.51370124]
(7, 449) [0.10427318 0.89572682]
(8, 23) [0.41698885 0.58301115]
(8, 73) [0.06518453 0.93481547]
(8, 83) [0.38818195 0.6